In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from xgboost import config_context
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
fert = pd.read_csv("Fertilizer Prediction.csv")

In [3]:
fert.head()

Temparature  Humidity   Moisture Soil Type  Crop Type  Nitrogen  Potassium  \
0           26         52        38     Sandy      Maize        37          0   
1           29         52        45     Loamy  Sugarcane        12          0   
2           34         65        62     Black     Cotton         7          9   
3           32         62        34       Red    Tobacco        22          0   
4           28         54        46    Clayey      Paddy        35          0   

   Phosphorous Fertilizer Name  
0            0            Urea  
1           36             DAP  
2           30        14-35-14  
3           20           28-28  
4            0            Urea

In [4]:
fert.describe()

Temparature  Humidity    Moisture   Nitrogen  Potassium  Phosphorous
count    99.000000  99.000000  99.000000  99.000000  99.000000    99.000000
mean     30.282828  59.151515  43.181818  18.909091   3.383838    18.606061
std       3.502304   5.840331  11.271568  11.599693   5.814667    13.476978
min      25.000000  50.000000  25.000000   4.000000   0.000000     0.000000
25%      28.000000  54.000000  34.000000  10.000000   0.000000     9.000000
50%      30.000000  60.000000  41.000000  13.000000   0.000000    19.000000
75%      33.000000  64.000000  50.500000  24.000000   7.500000    30.000000
max      38.000000  72.000000  65.000000  42.000000  19.000000    42.000000

In [5]:
fert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Temparature      99 non-null     int64 
 1   Humidity         99 non-null     int64 
 2   Moisture         99 non-null     int64 
 3   Soil Type        99 non-null     object
 4   Crop Type        99 non-null     object
 5   Nitrogen         99 non-null     int64 
 6   Potassium        99 non-null     int64 
 7   Phosphorous      99 non-null     int64 
 8   Fertilizer Name  99 non-null     object
dtypes: int64(6), object(3)
memory usage: 7.1+ KB


In [6]:
fert.isnull().sum()

Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64

In [7]:
# Distribution of target variable (Fertilizer Name)
plt.figure(figsize=(12, 6))
sns.countplot(y="Fertilizer Name", data=fert, order=fert['Fertilizer Name'].value_counts().index, palette="viridis")
plt.title("Distribution of Fertilizer Types")
plt.xlabel("Count")
plt.ylabel("Fertilizer Name")
plt.show()

In [8]:
fert_dict = {
    'Urea':1, 'DAP':2,'14-35-14':3, '28-28':4,'17-17-17':5,'20-20':6,'10-26-26':7
}

In [9]:
fert['fert_no'] = fert['Fertilizer Name'].map(fert_dict)

In [10]:
fert['fert_no'].value_counts()

fert_no
1    22
2    18
4    17
3    14
6    14
5     7
7     7
Name: count, dtype: int64

In [11]:
fert.drop('Fertilizer Name', axis=1,inplace=True)
fert.head()

Temparature  Humidity   Moisture Soil Type  Crop Type  Nitrogen  Potassium  \
0           26         52        38     Sandy      Maize        37          0   
1           29         52        45     Loamy  Sugarcane        12          0   
2           34         65        62     Black     Cotton         7          9   
3           32         62        34       Red    Tobacco        22          0   
4           28         54        46    Clayey      Paddy        35          0   

   Phosphorous  fert_no  
0            0        1  
1           36        2  
2           30        3  
3           20        4  
4            0        1

In [12]:
fert.select_dtypes(include=['object']).columns

Index(['Soil Type', 'Crop Type'], dtype='object')

In [13]:
le = LabelEncoder()
fert["Soil Type"] = le.fit_transform(fert['Soil Type'])
fert["Crop Type"] = le.fit_transform(fert['Crop Type'])

In [14]:
fert.head()

Temparature  Humidity   Moisture  Soil Type  Crop Type  Nitrogen  \
0           26         52        38          4          3        37   
1           29         52        45          2          8        12   
2           34         65        62          0          1         7   
3           32         62        34          3          9        22   
4           28         54        46          1          6        35   

   Potassium  Phosphorous  fert_no  
0          0            0        1  
1          0           36        2  
2          9           30        3  
3          0           20        4  
4          0            0        1

In [15]:
X = fert.drop('fert_no', axis=1)
y = fert['fert_no']
print(f"The shape of features is: {X.shape}")
print(f"The shape of target is: {y.shape}")

The shape of features is: (99, 8)
The shape of target is: (99,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
# Initialize the model with the parameters
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [19]:
accuracy = model.score(X_test, y_test)
print(f"The accuracy of the model is: {accuracy*100:.2f}%")

The accuracy of the model is: 100.00%


In [20]:
accuracy = model.score(X_train, y_train)
print(f"The accuracy of the model is: {accuracy*100:.2f}%")

The accuracy of the model is: 100.00%


In [22]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy is: {accuracy * 100 :.2f}%")
print()
print("Confusion Matrix is:\n", cm)
print()
print("Classification Report is : \n" , report)

Accuracy is: 100.00%

Confusion Matrix is:
 [[1 0 0 0 0 0 0]
 [0 3 0 0 0 0 0]
 [0 0 3 0 0 0 0]
 [0 0 0 4 0 0 0]
 [0 0 0 0 4 0 0]
 [0 0 0 0 0 3 0]
 [0 0 0 0 0 0 2]]

Classification Report is : 
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         4
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         2

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [23]:
def rf(Temperature, Humidity, Moisture, Soil_Type, Crop_Type, Nitrogen, Potassium, Phosphorous):
    features = np.array([[Temperature, Humidity, Moisture, Soil_Type, Crop_Type, Nitrogen, Potassium, Phosphorous]])
    transformed_features = sc.transform(features)
    prediction = model.predict(transformed_features).reshape(1, -1)
    fert_dict = {1:'Urea', 2:'DAP',3:'14-35-14', 4:'28-28',5:'17-17-17',6:'20-20',7:'10-26-26'}
    fertilizer = [fert_dict[i] for i in prediction[0]]

    return f"{fertilizer} is a best fertilizer for the given conditions"

In [24]:
Temperature = 26
Humidity = 0.5 
Moisture = 0.6
Soil_Type = 2
Crop_Type = 3
Nitrogen = 10
Potassium = 15 
Phosphorous = 6

rf(Temperature, Humidity, Moisture, Soil_Type, Crop_Type, Nitrogen, Potassium, Phosphorous)

"['17-17-17'] is a best fertilizer for the given conditions"

In [25]:
import pickle

# Save the model
with open("fertilizer.pkl", "wb") as f:
    pickle.dump(model, f)

# Load it later
with open("fertilizer.pkl", "rb") as f:
    model = pickle.load(f)